In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout
from keras.layers import Conv2D,MaxPooling2D,Flatten

Using TensorFlow backend.


In [2]:
df = pd.read_csv("train.csv")
df2=pd.read_csv("test.csv")
df.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
labels = df['label']
images= df.drop(columns=['label'])

In [4]:
trainData = images.values
trainLabels = labels.values
testData = df2.values
testData = testData.reshape(28000,28,28,1).astype('float32')/255
trainData = trainData.reshape(42000,28,28,1).astype('float32')/255
print(trainData.shape,testData.shape)

(42000, 28, 28, 1) (28000, 28, 28, 1)


In [5]:
model= Sequential()
model.add(Conv2D(64,(5,5),input_shape=(28,28,1),padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(10,activation="softmax"))

In [6]:
from keras.utils import np_utils
trainLabels = np_utils.to_categorical(trainLabels)
trainLabels.shape

(42000, 10)

In [7]:
from keras import optimizers
from keras import losses
model.compile(loss=losses.categorical_crossentropy,optimizer=optimizers.Adadelta(),metrics=['accuracy'])

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 64)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               409856    
__________

In [9]:
model.fit(trainData,
          trainLabels,
          batch_size=200,
          epochs=12,
          verbose=1
         )

Epoch 1/12
42000/42000 [==============================] - 118s 3ms/step - loss: 0.4160 - acc: 0.8659
Epoch 2/12
42000/42000 [==============================] - 121s 3ms/step - loss: 0.1004 - acc: 0.9705
Epoch 3/12
42000/42000 [==============================] - 116s 3ms/step - loss: 0.0729 - acc: 0.9784
Epoch 4/12
42000/42000 [==============================] - 122s 3ms/step - loss: 0.0571 - acc: 0.9836
Epoch 5/12
42000/42000 [==============================] - 127s 3ms/step - loss: 0.0485 - acc: 0.9860
Epoch 6/12
42000/42000 [==============================] - 119s 3ms/step - loss: 0.0422 - acc: 0.9872
Epoch 7/12
42000/42000 [==============================] - 117s 3ms/step - loss: 0.0387 - acc: 0.9887
Epoch 8/12
42000/42000 [==============================] - 118s 3ms/step - loss: 0.0337 - acc: 0.9900
Epoch 9/12
42000/42000 [==============================] - 116s 3ms/step - loss: 0.0308 - acc: 0.9907
Epoch 10/12
42000/42000 [==============================] - 120s 3ms/step - loss: 0.0283 - a

In [11]:
prediction = model.predict(testData)
prediction.shape

(28000, 10)

In [16]:
ans = prediction.argmax(axis=1)

In [23]:
res = pd.DataFrame(data=ans)
res.index+=1

In [24]:
res.to_csv("sol.csv",index=True,index_label="ImageId",header=['Label'])